In [1]:
# Parameters
image_path = "C:\\Users\\Administrator\\Desktop\\PID_PROJECT\\data\\test\\images\\test_pid.png"
outputs_dir = "C:\\Users\\Administrator\\Desktop\\PID_PROJECT\\outputs"
data_dir = "C:\\Users\\Administrator\\Desktop\\PID_PROJECT\\data\\dummy"


In [2]:
import os
from pathlib import Path
from ultralytics import YOLO
from functions.functions_OCR import *
import sys
sys.path.append('..')

pytesseract.pytesseract.tesseract_cmd = r"C:\venv\tesseract.exe"

outputs_dir = Path(outputs_dir)
detections_dir = outputs_dir / "detections"
detections_dir.mkdir(parents=True, exist_ok=True)



## 1. Use YOLO to identify PID symbols

In [3]:
# 1) Choose an image to process
# image_path = r'C:\Users\Administrator\Desktop\PID_PROJECT\data\test\images\test_pid.png'  # <-- test image
image_path = image_path  # Papermill 

img = cv2.imread(image_path)

In [4]:
# 2) Load model
model = YOLO('../models/pid_yolo.pt')  # from notebook_CV

# 3) Predict
results = model.predict(source=image_path, imgsz=640, conf=0.1, verbose=False)[0]

# 4) Build detections dataframe
rows = []
for b in results.boxes:
    cls_id = int(b.cls.item())
    conf   = float(b.conf.item())
    x1, y1, x2, y2 = map(float, b.xyxy[0].tolist())
    label = results.names[cls_id]
    cx, cy = (x1+x2)/2.0, (y1+y2)/2.0
    rows.append(dict(label=label, conf=conf, x1=x1, y1=y1, x2=x2, y2=y2, cx=cx, cy=cy))

df_det = pd.DataFrame(rows)

## OCR for detecting name near PID symbols

In [5]:
for r in rows:
    text, conf = read_tag_near_point(img, int(r["cx"]), int(r["cy"]), pad=83)
    r["name_ocr"] = text

df_det = pd.DataFrame(rows)

## Corrections for OCR, will be replaced with an LLM

In [6]:
# This will be later automated with an LLM, check functions_LLM
# df_after_llm = clean_pid_names_llm(df_det)

# Define OCR correction mapping
ocr_corrections = str.maketrans({
    'I': '1', 'l': '1',
    'O': '0', 'o': '0',
    'S': '5', 's': '5',
    'B': '8', 'G': '6'
})

def clean_name(row):
    name = row['name_ocr']
    
    # Fix OCR errors and keep only alphanumerics
    corrected = name.translate(ocr_corrections)
    corrected = re.sub(r'[^A-Za-z0-9]', '', corrected)
    
    # Replace first char based on label
    if row['label'] == 'valve':
        corrected = 'V' + corrected[1:]
    elif 'pressure' in row['label']:
        corrected = 'S' + corrected[1:]
    
    # Keep only 6 characters
    return corrected[:6]

df_det['name_ocr_clean'] = df_det.apply(clean_name, axis=1)

In [7]:
annot = img.copy()
for r in rows:
    x1,y1,x2,y2 = map(int, [r['x1'], r['y1'], r['x2'], r['y2']])
    cv2.rectangle(annot, (x1,y1), (x2,y2), (0,255,0), 2)
    cv2.putText(annot, f"{r['label']} {r['conf']:.2f}", (x1, max(0,y1-5)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)


base = Path(image_path).stem

base = os.path.basename(image_path).rsplit('.',1)[0]
# 5) Save detections for next notebooks
cv2.imwrite(str(detections_dir / f"{base}_annotated.png"), annot)
df_det.to_csv(detections_dir / f"{base}_detections.csv", index=False)